### Load dataset

- `titles:` list of titles of songs.
- `data:` list of dictionary contains title and path of songs.

In [ ]:
import pandas as pd

dataset = pd.read_csv('Data\Billboard.csv')
print(dataset.shape)
titles = list(dataset['Title'])
data = []

### Search method
- Use os to move songs into directory(Songs for popular data and Unpopular for unpoular data) contains all songs we saved it as mp3 files.
- Use requests to get songs from youtube.
- Use yt_dlp instead of youtube_dl to download songs because it faster than youtube_dl.

*`Parametres: `*
- `YDL_OPTIONS:` with format: 'bestaudio' to download best audio quality and noplaylist:True to avoid download playlists.
- `YoutubeDL(YDL_OPTIONS) as ydl:` download songs.
- `song:` this is a first song we get from youtube searching.
- `oldpath:` name of song.
- `newpath:` this is a path we want to move song into.
- `type:` to see if songs is poular or not.

    At the end we return song and path to save it in a dataframe.

In [ ]:
import os
from requests import get
# from youtube_dl import YoutubeDL (It take alot of time to download songs)
from yt_dlp import YoutubeDL # (It take no time)

YDL_OPTIONS = {'format': 'bestaudio', 'noplaylist':'True'}

def search(arg, type):
    with YoutubeDL(YDL_OPTIONS) as ydl:
        try:
            get(arg) 
        except:
            song = ydl.extract_info(f"ytsearch:{arg}", download=True)['entries'][0]
        else:
            song = ydl.extract_info(arg, download=True)
    
    oldpath = song['title'] + ' [' + song['id'] + '].' + song['ext']
    newpath = f"{type}/{song['title']}.mp3"
    os.rename(f"{oldpath}", newpath)

    return song, newpath

### Error would be happen
- *If error is 403:* you must update i with last value of i and run this cell again.
- *Path doesn't exist:* go to name of this song and rename it with .mp3 finally move it to his directory and update data with new dict.

### Now download all songs
    search for song by title and name of artist to avoid downloaded another title from youtube.
*Parameters*
- `i:` iterator for looping in every title in titles.
- `string:` title of song and name of artist to search with it to avoid download anything else song.
- in search method when i from 0 to 380 is popular(Songs) and from 381 to 785 is unpopular(Unpopuplar).

In [ ]:
i = 0
while i < len(titles):
    string = titles[i] + ' ' + dataset['Artist'][dataset['Title'] == titles[i]].values[0]
    if i < 381:
        audio, path = search(string, 'Songs') # Songs directory to collect all popular songs.
    else:
        audio, path = search(string, 'Unpopular') # Unpopular directory to collect all unpopular songs.

    data.append({'Title': titles[i], 'Path': path})
    i += 1
    if i == len(titles):
        break

In [ ]:
newData = pd.read_csv('NewData.csv')
for row in data:
    newData = newData.append(row, ignore_index = True)

In [ ]:
newData

In [ ]:
newData.to_csv('Data\NewData.csv', index=False)

### Now add this newData to orgnial dataset as a columns and name it with DataWithMusic

In [ ]:
# Add path column
dataset['Path'] = newData['Path']
# Add title column with Title column in newData to check data
dataset['title'] = newData['Title']

In [ ]:
newtitles = list(dataset['title'])  # to collect titles which we added
titles = list(dataset['Title'])     # to coolect old Titles

In [ ]:
lostdata = []  # list to Titles and titles with doesn't matches
i = 0
for i in range(len(titles)):
    if titles[i] != newtitles[i]:
        lostdata.append({'Title':titles[i],'title':newtitles[i]})
    i+=1

# to see if any lost data or not. if(just delete rows) else dataset is good
print(len(lostdata))
lostdata

In [ ]:
# Here I found this data was lost so I dropped it.
'''
first = dataset[dataset['Title']=='Can\'t Stop The Feeling!']
second = dataset[dataset['Title']=='Heathens']
third = dataset[dataset['title']=='How Do You Sleep?']
print(first)
print(second)
print(third)
dataset = dataset.drop(109)
dataset = dataset.drop(111)
dataset = dataset.drop(69)
'''

In [ ]:
# if we deleted rows just reset index after that delete title row
dataset.reset_index()
del dataset['title']

In [ ]:
# Save last data
dataset.to_csv('Data\DataWithMusic.csv', index=False)